In [51]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv("globalterrorismdb_0718dist.csv", encoding="utf-8", encoding_errors="replace")

C:\Users\mbans\AppData\Local\Temp\ipykernel_6476\2012313502.py:1: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("globalterrorismdb_0718dist.csv", encoding="utf-8", encoding_errors="replace")


In [3]:
df = df.drop([4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121])

In [5]:
print(df.columns.values)

['eventid' 'iyear' 'imonth' 'iday' 'approxdate' 'extended' 'resolution'
 'country' 'country_txt' 'region' 'region_txt' 'provstate' 'city'
 'latitude' 'longitude' 'specificity' 'vicinity' 'location' 'summary'
 'crit1' 'crit2' 'crit3' 'doubtterr' 'alternative' 'alternative_txt'
 'multiple' 'success' 'suicide' 'attacktype1' 'attacktype1_txt'
 'attacktype2' 'attacktype2_txt' 'attacktype3' 'attacktype3_txt'
 'targtype1' 'targtype1_txt' 'targsubtype1' 'targsubtype1_txt' 'corp1'
 'target1' 'natlty1' 'natlty1_txt' 'targtype2' 'targtype2_txt'
 'targsubtype2' 'targsubtype2_txt' 'corp2' 'target2' 'natlty2'
 'natlty2_txt' 'targtype3' 'targtype3_txt' 'targsubtype3'
 'targsubtype3_txt' 'corp3' 'target3' 'natlty3' 'natlty3_txt' 'gname'
 'gsubname' 'gname2' 'gsubname2' 'gname3' 'gsubname3' 'motive'
 'guncertain1' 'guncertain2' 'guncertain3' 'individual' 'nperps'
 'nperpcap' 'claimed' 'claimmode' 'claimmode_txt' 'claim2' 'claimmode2'
 'claimmode2_txt' 'claim3' 'claimmode3' 'claimmode3_txt' 'compclaim'


In [6]:
df.success.unique()

array([1, 0], dtype=int64)

In [10]:
df.crit1.unique()

array([1, 0], dtype=int64)

In [ ]:
df2 = df.loc[:, ['iyear', 'imonth', 'iday', 'approxdate', ]]

In [11]:
df.resolution.unique()

array([nan, '3/8/1970', '3/15/1970', ..., '3/23/2013', '12/9/2011',
       '12/27/2011'], dtype=object)

In [89]:
df3 = df.select_dtypes(include=np.number)

In [90]:
df3 = df3.iloc[:, 11:24]

In [91]:
df3

,crit1,crit2,crit3,doubtterr,alternative,multiple,success,suicide,attacktype1,attacktype2,attacktype3,targtype1,targsubtype1
0,1,1,1,0.0,NaN,0.0,1,0,1,NaN,NaN,14,68.0
1,1,1,1,0.0,NaN,0.0,1,0,6,NaN,NaN,7,45.0
2,1,1,1,0.0,NaN,0.0,1,0,1,NaN,NaN,10,54.0
3,1,1,1,0.0,NaN,0.0,1,0,3,NaN,NaN,7,46.0
5,1,1,1,0.0,NaN,0.0,1,0,2,NaN,NaN,3,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,1,1,0,1.0,1.0,0.0,1,0,2,NaN,NaN,4,36.0
181687,1,1,0,1.0,1.0,0.0,1,0,3,NaN,NaN,4,27.0
181688,1,1,1,0.0,NaN,0.0,1,0,7,NaN,NaN,14,76.0
181689,1,1,1,0.0,NaN,0.0,0,0,3,NaN,NaN,2,21.0


In [93]:
df3 = df3.drop(['alternative', 'attacktype2', 'attacktype3'], axis=1).dropna()

In [94]:
df3.shape

(171300, 10)

In [36]:
df3.drop(['attacktype1'], axis=1, inplace=True)

In [96]:
df3.reset_index(drop=True, inplace=True)

In [97]:
y = df3.loc[:, 'success'].to_numpy()
X = df3.drop(['success'], axis=1).to_numpy()

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [102]:
dt = DecisionTreeClassifier(max_depth=10, random_state=42)
dt.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=10, random_state=42)

In [103]:
y_hat = dt.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print(acc)

clf_rep = classification_report(y_test, y_hat)
print(clf_rep)


cm = confusion_matrix(y_test, y_hat)
print(cm)

0.9096614127262114
              precision    recall  f1-score   support

           0       0.55      0.03      0.06      3116
           1       0.91      1.00      0.95     31144

    accuracy                           0.91     34260
   macro avg       0.73      0.52      0.51     34260
weighted avg       0.88      0.91      0.87     34260

[[  106  3010]
 [   85 31059]]


In [101]:
# Grid-search for finding optimal hyperparameter
max_depth_range = np.array(range(1, 20))

param_grid = {'max_depth': max_depth_range}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_max_depth = grid_search.best_params_['max_depth']
print(f"Best max_depth: {best_max_depth}")

Best max_depth: 10
